Evan Edelstein
EN.605.645.82.SP26

# Module 5 - Programming Assignment

## Directions

1. Change the name of this file to be your JHED id as in `jsmith299.ipynb`. Because sure you use your JHED ID (it's made out of your name and not your student id which is just letters and numbers).
2. Make sure the notebook you submit is cleanly and fully executed. I do not grade unexecuted notebooks.
3. Submit your notebook back in Blackboard where you downloaded this file.

*Provide the output **exactly** as requested*

## Solving Normal Form Games

In [1]:
from typing import List, Tuple, Dict, Callable, Set

In the lecture we talked about the Prisoner's Dilemma game, shown here in Normal Form:

Player 1 / Player 2  | Defect | Cooperate
------------- | ------------- | -------------
Defect  | -5, -5 | -1, -10
Cooperate  | -10, -1 | -2, -2

where the payoff to Player 1 is the left number and the payoff to Player 2 is the right number. We can represent each payoff cell as a Tuple: `(-5, -5)`, for example. We can represent each row as a List of Tuples: `[(-5, -5), (-1, -10)]` would be the first row and the entire table as a List of Lists:

In [2]:
prisoners_dilemma = [[(-5, -5), (-1, -10)], [(-10, -1), (-2, -2)]]

prisoners_dilemma

[[(-5, -5), (-1, -10)], [(-10, -1), (-2, -2)]]

in which case the strategies are represented by indices into the List of Lists. For example, `(Defect, Cooperate)` for the above game becomes `prisoners_dilemma[ 0][ 1]` and returns the payoff `(-1, -10)` because 0 is the first row of the table ("Defect" for Player 1) and 1 is the 2nd column of the row ("Cooperate" for Player 2).

For this assignment, you are going write a function that uses Successive Elimination of Dominated Strategies (SEDS) to find the **pure strategy** Nash Equilibrium of a Normal Form Game. The function is called `solve_game`:

```python
def solve_game( game: List[List[Tuple]], weak=False) -> List[Tuple]:
    pass # returns strategy indices of Nash equilibrium or None.
```

and it takes two parameters: the game, in a format that we described earlier and an optional boolean flag that controls whether the algorithm considers only **strongly dominated strategies** (the default will be false) or whether it should consider **weakly dominated strategies** as well.

It should work with game matrices of any size and it will return the **strategy indices** of the Nash Equilibrium. If there is no **pure strategy** equilibrium that can be found using SEDS, return the empty List (`[]`).


<div style="background: mistyrose; color: firebrick; border: 2px solid darkred; padding: 5px; margin: 10px;">
Do not return the payoff. That's not useful. Return the strategy indices, any other output is incorrect.
</div>

As before, you must provide your implementation in the space below, one Markdown cell for documentation and one Code cell for implementation, one function and assertations per Codecell.


---

<a id="is_strongly_dominated"></a>
## is_strongly_dominated

*`is_strongly_dominated` Determine if strategy C is strictly dominated by strategy j. That means each payoff in j is greater than C.* **Used by**: [is_dominated](#is_dominated)

* **payoffs_c** List[int] - list of strategy payoff 
* **payoffs_j** List[int] - list of strategy payoff 

**returns** bool - is strategy C is strictly dominated by strategy j

In [ ]:
def is_strongly_dominated(payoffs_c: List[int], payoffs_j: List[int]) -> bool:
    return all(p1 < p2 for p1, p2 in zip(payoffs_c, payoffs_j))

In [ ]:
assert is_strongly_dominated([1,1,1],[2,2,2]) is True # test 1 - dominated
assert is_strongly_dominated([5, 6, 7], [2, 3, 4]) is False # test 2- not dominated
assert is_strongly_dominated([1,1,1],[1,1,1]) is False # test 3 - equal payoff should not be dominated

<a id="is_weakly_dominated"></a>
## is_weakly_dominated

*`is_weakly_dominated` Determine if strategy C is weakly dominated by strategy j. That means each payoff in j is greater than or equal to C and at least one payoff is greater.* **Used by**: [is_dominated](#is_dominated)

* **payoffs_c** List[int] - list of strategy payoff 
* **payoffs_j** List[int] - list of strategy payoff 

**returns** bool - is strategy C is weakly dominated by strategy j

In [ ]:
def is_weakly_dominated(payoffs_c: List[int], payoffs_j: List[int]) -> bool: 
    return all([p1 <= p2 for p1, p2 in zip(payoffs_c, payoffs_j)]) and any([p1 < p2 for p1, p2 in zip(payoffs_c, payoffs_j)])

In [ ]:
assert is_weakly_dominated([1,1,1],[1,1,2]) is True # test 1 - dominated
assert is_weakly_dominated([5, 6, 7], [2, 3, 4]) is False # test 2- not dominated
assert is_weakly_dominated([1,1,1],[1,1,1]) is False # test 3 - equal payoff should not be dominated

<a id="is_dominated"></a>
## is_dominated

*`is_dominated` Determine if strategy C is strongly or weakly dominated by strategy j.* **Used by**: [get_dominated_strategies](#get_dominated_strategies)

* **payoffs_c** List[int] - list of strategy payoff 
* **payoffs_j** List[int] - list of strategy payoff 
* **weak** bool - if true check for weak dominance else check for strict dominance


**returns** bool - is strategy C is dominated by strategy j

In [ ]:
def is_dominated(payoffs_c: List[int], payoffs_j: List[int], weak: bool) -> bool:
    if weak:
        return is_weakly_dominated(payoffs_c, payoffs_j)
    return is_strongly_dominated(payoffs_c, payoffs_j)

In [ ]:
assert is_dominated([1,1,1],[2,2,2], False) is True # test 1a - dominated
assert is_dominated([5, 6, 7], [2, 3, 4], False) is False # test 2a- not dominated
assert is_dominated([1,1,1],[1,1,1], False) is False # test 3a - equal payoff should not be dominated

assert is_dominated([1,1,1],[1,1,2], True) is True # test 1b - dominated
assert is_dominated([5, 6, 7], [2, 3, 4], True) is False # test 2b- not dominated
assert is_dominated([1,1,1],[1,1,1], True) is False # test 3b - equal payoff should not be dominated

<a id="get_payoffs"></a>
## get_payoffs

*`get_payoffs` get a players payoffs for a specified strategy.* **Used by**: [get_dominated_strategies](#get_dominated_strategies)

* **game** List[List[Tuple[int, int]]] - payoff matrix in normal form, represented as nested list of tuples. 
* **player** int - player to get payoffs for
* **strategy** int - row/col to get payoffs of
* **indices**  Tuple[int, ...] - tuple of non-eliminated rows/cols


**returns** List[int] - list of payoffs for the player choosing a strategy

In [7]:
def get_payoffs(game: List[List[Tuple[int, int]]], player: int, strategy: int, indices: Tuple[int, ...]) -> List[int]:
    return [game[strategy][c][player] for c in indices] if player == 0 else [game[r][strategy][player] for r in indices] if player == 1 else []

In [ ]:
game = [[(1, 2), (3, 4)], [(5,6), (7, 8)]]

assert get_payoffs(game, 0, 0, (0,1)) == [1,3]  # test 1 - get player 0's payoffs
assert get_payoffs(game, 1, 1, (0,1)) == [4, 8] # test 2 - get player 1's payoffs
assert get_payoffs(game, 4, 0, (0,1)) == [] # test 3 - unknown player returns empty list

<a id="remove_strategy"></a>
## remove_strategy

*`remove_strategy` remove a strategy from a tuple of strategies.* **Used by**: [successors](#successors)

* **indices**  Tuple[int, ...] - tuple of non-eliminated rows/cols
* **index**  int - index of strategy to remove from indices


**returns** Tuple[int, ...] - indices with strategy at index removed

In [44]:
def remove_strategy(indices, index):
    return tuple(i for i in indices if i != index) if isinstance(indices, tuple) else tuple()

In [45]:
assert remove_strategy((1, 2, 3), 1) == (2,3) # test 1 - normal 
assert remove_strategy((1), 1 ) == () # test 2 - remove single tuple returns empty tuple
assert remove_strategy((1, 2, 3), 4) == (1, 2, 3) # test 3 - removal of index outside of tuple returns input tuple

<a id="get_dominated_strategies"></a>
## get_dominated_strategies

*`get_dominated_strategies` find dominated strategies in the game for a specified player. * **Used by**: [successors](#successors)

* **game** List[List[Tuple[int, int]]] - payoff matrix in normal form, represented as nested list of tuples. 
* **player** int - player to get payoffs for
* **row_indices**  Tuple[int, ...] - tuple of non-eliminated rows/cols
* **col_indices**  Tuple[int, ...] - tuple of non-eliminated rows/cols
* **weak** bool - if true check for weak dominance else check for strict dominance

**returns** Set[int]- list of payoffs for the player choosing a strategy

In [ ]:
def get_dominated_strategies(game: List[List[Tuple[int, int]]], player: int, row_indices: Tuple[int, ...], col_indices: Tuple[int, ...], weak: bool) -> Set[int]:
    indices = row_indices if player == 0 else col_indices
    opponent_indices = col_indices if player == 0 else row_indices

    dominated_strategies = set()
    for c in indices:
        payoffs_c = get_payoffs(game, player, c, opponent_indices)
        for j in indices:
            if j != c and is_dominated(payoffs_c, get_payoffs(game, player, j, opponent_indices), weak):
                dominated_strategies.add(c)
                break
    return dominated_strategies


In [ ]:
def successors(game: List[List[Tuple[int, int]]], strategies: Tuple[Tuple[int, ...], Tuple[int, ...]], weak: bool) -> List[Tuple[Tuple[int, ...], Tuple[int, ...]]]:
    valid_rows, valid_cols= strategies[0], strategies[1]
    successor_strategies = []
    dominated_rows = get_dominated_strategies(game, 0, valid_rows, valid_cols, weak)
    for row in dominated_rows:
        s = remove_strategy(valid_rows, row)
        successor_strategies.append((s, valid_cols))
        

    dominated_cols = get_dominated_strategies(game, 1, valid_rows, valid_cols, weak)
    for col in dominated_cols:
        s = remove_strategy(valid_cols, col)
        successor_strategies.append((valid_rows, s))
    return successor_strategies


In [11]:
def add_strategy_to_solution(strategies: Tuple[Tuple[int, ...], Tuple[int, ...]], solutions: Set[Tuple[int, int]]):
    for i in strategies[0]:
        for j in strategies[1]:
            solutions.add((i, j))
    return solutions

In [12]:
def add_strategies_to_frontier(next_strategies: List[Tuple[Tuple[int, ...], Tuple[int, ...]]], frontier: Set[Tuple[Tuple[int, ...], Tuple[int, ...]]], explored: Set[Tuple[Tuple[int, ...], Tuple[int, ...]]]):
    for child in next_strategies:
        if child not in explored:
            frontier.add(child)

## solve_game


In [13]:
def solve_game(game: List[List[Tuple[int, int]]], weak: bool = False) -> List[Tuple[int, int]]:
    rows, cols = len(game), len(game[0])
    solutions: Set[Tuple[int, int]] = set()
    initial_strategies: Tuple[Tuple[int, ...], Tuple[int, ...]] = (tuple(range(rows)),tuple(range(cols)))
    frontier: Set[Tuple[Tuple[int, ...], Tuple[int, ...]]] = {initial_strategies}
    visited: Set[Tuple[Tuple[int, ...], Tuple[int, ...]]] = set()

    while frontier:
        strategies = frontier.pop()
        visited.add(strategies)
        next_strategies = successors(game, strategies, weak)
        if not next_strategies and len(strategies[0]) < rows and len(strategies[1]) < cols:
            add_strategy_to_solution(strategies, solutions)
        else:
            add_strategies_to_frontier(next_strategies, frontier, visited)
    return list(solutions)

In [14]:
prisoners_dilemma = [[(-5, -5), (-1, -10)], [(-10, -1), (-2, -2)]]
game2 = [
    [(1, 0), (3, 1), (1, 1)],
    [(1, 1), (3, 0), (0, 1)],
    [(2, 2), (3, 3), (0, 2)],
]
game3 = [
    [(10, 10), (14, 12), (14, 15)],
    [(12, 14), (20, 20), (28, 15)],
    [(15, 14), (15, 28), (25, 25)],
]

assert solve_game(prisoners_dilemma, False) == [(0, 0)]
assert solve_game(game2, True) == [(0, 1), (0, 2), (2, 1)]
assert solve_game(game3, False) == [(1, 1)]

## Additional Directions

Create three games as described and according to the following:

1. Your games must be created and solved "by hand".
2. The strategy pairs must **not** be on the main diagonal (0, 0), (1, 1), or (2, 2). And the solution cannot be the same for both Game 1 and Game 2.
3. Make sure you fill out the Markdown ("?") with your game as well as the solution ("?").
4. Remember, **do not return the payoff**, return the strategy indices (a list of them).

## Before you code...

Solve the following game by hand using SEDS and weakly dominated strategies. 
The game has two (pure) Nash Equilibriums. 
You should find all of them.
This will help you think about what you need to implement to make the algorithm work.
**Hint**: You will need State Space Search from Module 1 and SEDS from Module 5 to get the full algorithm to work.

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 1/0 | 3/1 | 1/1 |
|1  | 1/1 | 3/0 | 0/1 |
|2  | 2/2 | 3/3 | 0/2 |


**Solutions**: 
- Turn 1: 
    - Player 1: compare strategies 0 (1, 3, 1), 1 (1, 3, 0) and 2 (2, 3, 0). We see strategy 1 is weekly dominated by strategy 0. 
    - remove row 1:

        | Player 1 / Player 2  | 0 | 1 | 2 |
        |----|----|----|----|
        |0  | 1/0 | 3/1 | 1/1 |
        |2  | 2/2 | 3/3 | 0/2 |

- Turn 2:
    - Player 2: compare strategies 0 (0, 2), 1 (1, 3) and 2 (1, 2). We see strategy 0 is weekly dominated by strategy 1.
    - remove col 0:
        
        | Player 1 / Player 2  | 1 | 2 |
        |----|----|----|
        |0  | 3/1 | 1/1 |
        |2  | 3/3 | 0/2 |

- Turn 3:
    - Player 1: compare strategies 0 (3, 1) and 2 (3, 0). We see strategy 0 is dominated by strategy 2.
    - remove row 0:

        | Player 1 / Player 2  | 1 | 2 |
        |----|----|----|
        |0  | 3/1 | 1/1 |
    
- Since there are no more eliminations to make the Nash Equilibriums are strategies (0,1) and (0,2).

### Test Game 1. Create a 3x3 two player game

**that can only be solved using the Successive Elimintation of Strongly Dominated Strategies**

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 3, 0 | 3,3 | 3,1 |
|1  | 1,0 | 1,3 | 1,1 |
|2  | 0,0 | 0,3 | 0,1 |

**Solution:**? (strategy indices)

In [15]:
test_game_1 = [
    [(0, 0), (0, 3), (0, 1)],
    [(1, 0), (1, 3), (1, 1)],
    [(3, 0), (3, 3), (3, 1)], 
]

solution = solve_game(test_game_1)
print(solution)

[(2, 1)]


In [16]:
assert solution == [(2,1)]  # insert your solution from above.

### Test Game 2. Create a 3x3 two player game

**that can only be solved using the Successive Elimintation of Weakly Dominated Strategies**

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 2,2 | 2,4 | 0,2 |
|1  | 2,2 | 2,1 | 1,1 |
|2  | 2,3 | 3,3 | 1,2 |

**Solution:**? (strategy indices)

In [17]:
test_game_2 = [
    [(2,2), (2,4), (0,2)],
    [(2,2), (2,1), (1,1)],
    [(2,3), (3,3), (1,2)]
]

strong_solution = solve_game(test_game_2)
weak_solution = solve_game(test_game_2, weak=True)

In [18]:
assert strong_solution == [] 
assert weak_solution == [(1,0), (2,0), (2,1)]  # insert your solution from above.

### Test Game 3. Create a 3x3 two player game

**that cannot be solved using the Successive Elimintation of Dominated Strategies at all**

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | ? | ? | ? |
|1  | ? | ? | ? |
|2  | ? | ? | ? |

**Solution:** None

In [19]:
test_game_3 = [
    [(1,1), (0,2), (2,0)], 
    [(0,2), (2,0), (1,1)], 
    [(2,0), (1,1), (0,2)]
]

strong_solution = solve_game(test_game_3)
weak_solution = solve_game(test_game_3, weak=True)

In [20]:
assert strong_solution == []
assert weak_solution == []

### Test Game 4. Multiple Equilibria

You solved the following game by hand, above.
Now use your code to solve it.

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 1/0 | 3/1 | 1/1 |
|1  | 1/1 | 3/0 | 0/1 |
|2  | 2/2 | 3/3 | 0/2 |

**Solutions:** (copy from above)

In [21]:
test_game_4 = [[(1, 0), (3, 1), (1, 1)], [(1, 1), (3, 0), (0, 1)], [(2, 2), (3, 3), (0, 2)]]

strong_solution = solve_game(test_game_4)
weak_solution = solve_game(test_game_4, weak=True)

In [22]:
assert strong_solution == []
assert weak_solution == [(0,1), (0,2), (2,1)]  # put solution here

## Before You Submit...

1. Did you provide output exactly as requested? **Don't forget to fill out the Markdown tables with your games**.
2. Did you re-execute the entire notebook? ("Restart Kernel and Rull All Cells...")
3. If you did not complete the assignment or had difficulty please explain what gave you the most difficulty in the Markdown cell below.
4. Did you change the name of the file to `jhed_id.ipynb`?

Do not submit any other files.